In [1]:
# Here's several helpful packages to load

import numpy as np
import pandas as pd 
import math
from sklearn import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jee-advanced-2022-opening-closing-ranks/IIT_data.csv


In [2]:
df=pd.read_csv('/kaggle/input/jee-advanced-2022-opening-closing-ranks/IIT_data.csv')
df.shape

(2803, 10)

In [3]:
df.drop_duplicates(inplace=True)
df = df.dropna()
df = df.drop(['Year'], axis=1)

In [4]:
df.shape

(2799, 9)

In [5]:
df = df.drop(['Sr.'], axis=1)
df.insert(0, 'Sr.', np.arange(df.shape[0]))
df['Sr.'] = df['Sr.'].astype(int)

In [6]:
df

,Sr.,Institute,Program,Duration,Degree,Category,Pool,Opening,Closing
0,0,IIT-Bhubaneswar,Civil Engineering,4 Years,B.Tech,GEN,Gender-Neutral,9193,13624
1,1,IIT-Bhubaneswar,Civil Engineering,4 Years,B.Tech,GEN,Female-Only,16138,21674
2,2,IIT-Bhubaneswar,Civil Engineering,4 Years,B.Tech,GEN-EWS,Gender-Neutral,1907,2171
3,3,IIT-Bhubaneswar,Civil Engineering,4 Years,B.Tech,GEN-EWS,Female-Only,3334,3334
4,4,IIT-Bhubaneswar,Civil Engineering,4 Years,B.Tech,OBC-NCL,Gender-Neutral,4094,4615
...,...,...,...,...,...,...,...,...,...
2798,2794,IIT-Dharwad,Mechanical Engineering,4 Years,B.Tech,OBC-NCL,Female-Only,7459,8884
2799,2795,IIT-Dharwad,Mechanical Engineering,4 Years,B.Tech,SC,Gender-Neutral,1709,2379
2800,2796,IIT-Dharwad,Computer Science and Engineering,4 Years,B.Tech,GEN-EWS,Female-Only,1217,1217
2801,2797,IIT-Dharwad,Computer Science and Engineering,4 Years,B.Tech,OBC-NCL,Gender-Neutral,1378,1848


In [7]:
for i in range(2799):
    for j in range(2):
        x = df.iloc[i,7+j]
        if(x[-1]=='P'):
            df.iloc[i,7+j]=int(x[:-4])
        else:
            df.iloc[i,7+j]=int(x)

In [8]:
model_df = df.drop(['Institute','Duration','Degree'], axis=1)
label_encoder = preprocessing.LabelEncoder()

model_df['Program']= label_encoder.fit_transform(model_df['Program'])
d3 = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
model_df['Category']= label_encoder.fit_transform(model_df['Category'])
d1 = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

model_df['Pool']= label_encoder.fit_transform(model_df['Pool'])

d2 = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

d1

{'GEN': 0,
 'GEN-EWS': 1,
 'GEN-EWS-PWD': 2,
 'GEN-PWD': 3,
 'OBC-NCL': 4,
 'OBC-NCL-PWD': 5,
 'SC': 6,
 'SC-PWD': 7,
 'ST': 8,
 'ST-PWD': 9}

In [9]:
d2

{'Female-Only': 0, 'Gender-Neutral': 1}

In [10]:
X = model_df.drop(['Sr.','Program'], axis=1)
y = model_df[['Sr.','Program']]
y

,Sr.,Program
0,0,26
1,1,26
2,2,26
3,3,26
4,4,26
...,...,...
2798,2794,71
2799,2795,71
2800,2796,33
2801,2797,33


**DEFINING MY OWN SCORER**

In [11]:
def my_scorer_func(test_y, y_pred):
    y_predn=[]
    for i in range(test_y.shape[0]):
        y_predn.append(df.iloc[int(y_pred[i][0])][9])
    y_predn = np.array(y_predn, dtype='float32')
    test_y = test_y.to_numpy()
    y_testn=[]
    for i in range(test_y.shape[0]):
        y_testn.append(df.iloc[int(test_y[i][0])][9])
    y_testn = pd.Series(y_testn) 
    y_testn
    from sklearn.metrics import r2_score
    res = r2_score(y_testn, y_predn)
    return res
from sklearn.metrics import make_scorer
my_scorer = make_scorer(my_scorer_func, greater_is_better=True)

In [12]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state=42)

In [13]:
# Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 10, stop = 10, num = 1)]
# Number of features to consider at every split
#max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
#max_depth = [5,8]
# Minimum number of samples required to split a node
#min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
#bootstrap = [True, False]

In [14]:
# Create the param grid
# param_grid = {#'n_estimators': n_estimators,
               #'max_features' : max_features,
               #'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
               #'bootstrap': bootstrap
             #}
# print(param_grid)

In [15]:
# from sklearn.ensemble import RandomForestClassifier
# rf_Model = RandomForestClassifier()

In [16]:
# from sklearn.model_selection import GridSearchCV
# rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid,scoring=my_scorer)

In [17]:
# rf_Grid.fit(train_X, train_y)

In [18]:
# rf_Grid.best_params_

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=80, max_depth=5, min_samples_split=2, min_samples_leaf=1)
rf.fit(X, y)

RandomForestClassifier(max_depth=5, n_estimators=80)

In [20]:
#rf1 = RandomForestClassifier(n_estimators=100, max_depth=5)
#rf1.fit(X, y)

In [21]:
#y_pred = rf.predict(test_X)

In [22]:
##creating local test-df
test_df = pd.DataFrame()
test_df['Category']=[0]
test_df['Pool']=[1]
test_df['Opening']=[500]
test_df['Closing']=[700]

rf.predict(test_df)

array([[1283,   33]])

In [23]:
df.iloc[2180]

Sr.                                      2180
Institute                  IIT-(BHU) Varanasi
Program      Computer Science and Engineering
Duration                              4 Years
Degree                                 B.Tech
Category                                  GEN
Pool                           Gender-Neutral
Opening                                   426
Closing                                   897
Name: 2183, dtype: object

In [24]:
gen_df = test_df
predictions = rf.predict_proba(test_df)[0]
res = np.argsort(predictions)[-20:]
j=0
i=0
while((j<10) and (i<20)):
    if(d1[df.iloc[res[0][-1-i]][5]]==test_df.iloc[0][0]):
        if(d2[df.iloc[res[0][-1-i]][6]]==test_df.iloc[0][1]):
            print(df.iloc[res[0][-1-i]][1:9])
            j=j+1
    i=i+1

Institute                                 IIT-Roorkee
Program      Data Science and Artificial Intelligence
Duration                                      4 Years
Degree                                         B.Tech
Category                                          GEN
Pool                                   Gender-Neutral
Opening                                           535
Closing                                           822
Name: 1761, dtype: object
Institute                                        IIT-Kharagpur
Program      Electronics and Electrical Communication Engin...
Duration                                               4 Years
Degree                                                  B.Tech
Category                                                   GEN
Pool                                            Gender-Neutral
Opening                                                    434
Closing                                                   1145
Name: 860, dtype: object
Institute    

In [25]:
import pickle
with open('clg_prd.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [26]:
#os.remove("/kaggle/working/clg_prd.pkl")